In [2]:
import pathlib
import os
import subprocess
import time
from tqdm import tqdm
import re

# Configuración de paths
current_script = pathlib.Path().resolve()
script_dir = current_script.parent

path_generador = script_dir / "generate_problem.py"
path_dominio = script_dir / "3_2/dron_domain.pddl"
path_problema = script_dir / "3_2/Problemas"
path_planificador = script_dir.parent / "planificadores/optic-clp"
path_planes = script_dir / "3_2/planes_generados"
path_problema.mkdir(exist_ok=True)
path_planes.mkdir(exist_ok=True)
max_time = 60
size = 2
max_size = 0

concurrency_pattern = re.compile(r"\(\s*\(.+?\)\s+\(.+?\)\s*\)")

pbar = tqdm(desc="Testing problem sizes", unit="size")
while True:
    # Generar el archivo de problema
    gen_cmd = [
        "python3", str(path_generador),
        "-d", "2", "-r", "2",
        "-l", str(size), "-p", str(size),
        "-c", str(size), "-g", str(size),
        "-o", str(path_problema)
    ]
    try:
        gen_result = subprocess.run(gen_cmd, check=True, capture_output=True, text=True)
    except subprocess.CalledProcessError as e:
       
        break

    # Extraer el nombre del archivo generado
    problem_file = None
    for line in gen_result.stdout.splitlines():
        if "writing to" in line:
            problem_file = pathlib.Path(line.split("writing to")[-1].strip())
            break
    if problem_file is None:
        problem_file = path_problema / f"drone_problem_d2_r2_l{size}_p{size}_c{size}_g{size}_ct2.pddl"

    # Ejecutar el planificador
    optic_cmd = [
        str(path_planificador), '-E', '-N',
        str(path_dominio),
        str(problem_file)
    ]
    start_time = time.time()
    try:
        result = subprocess.run(
            optic_cmd,
            timeout=max_time,
            capture_output=True,
            text=True
        )
        execution_time = time.time() - start_time
        if result.returncode == 0:
            # Detectar acciones concurrentes
            concurrent_actions = any(
                concurrency_pattern.search(line)
                for line in result.stdout.split('\n')
            )
            # Guardar el plan en la carpeta planes_generados
            plan_filename = path_planes / f"plan_size_{size}.txt"
            with open(plan_filename, "w") as f:
                f.write(result.stdout)
            pbar.write(
                f"Size {size}: Solved in {execution_time:.2f}s | "
                f"Concurrency: {concurrent_actions}"
            )
            max_size = size
            size += 1
        else:
            pbar.write(f"Planner failed at size {size} with return code {result.returncode}")
            break

    except subprocess.TimeoutExpired:
        pbar.write(f"Timeout at size {size}")
        break
    except Exception as e:
        print("Unexpected error:", e)
        break

    pbar.update(1)

pbar.close()
print(f"\nMaximum solvable size: {max_size}")


Testing problem sizes: 1size [03:32, 212.06s/size]

                                          

Size 2: Solved in 7.17s | Concurrency: False



Testing problem sizes: 1size [01:07, 67.28s/size]

Timeout at size 3

Maximum solvable size: 2


In [10]:
def test_single_problem(size=2, d=2, r=2, l=None, p=None, c=None, g=None, max_time=60):
    # Usar valores por defecto si no se especifican
    l = l if l is not None else size
    p = p if p is not None else size
    c = c if c is not None else size
    g = g if g is not None else size

    # Generar el archivo de problema
    gen_cmd = [
        "python3", str(path_generador),
        "-d", str(d), "-r", str(r),
        "-l", str(l), "-p", str(p),
        "-c", str(c), "-g", str(g),
        "-o", str(path_problema)
    ]
    try:
        gen_result = subprocess.run(gen_cmd, check=True, capture_output=True, text=True)
    except subprocess.CalledProcessError as e:
        print("Error generating problem:", e)
        return

    # Extraer el nombre del archivo generado
    problem_file = None
    for line in gen_result.stdout.splitlines():
        if "writing to" in line:
            problem_file = pathlib.Path(line.split("writing to")[-1].strip())
            break
    if problem_file is None:
        problem_file = path_problema / f"drone_problem_d{d}_r{r}_l{l}_p{p}_c{c}_g{g}_ct2.pddl"

    # Ejecutar el planificador
    optic_cmd = [
        str(path_planificador), '-E', '-N',
        str(path_dominio),
        str(problem_file)
    ]
    start_time = time.time()
    try:
        result = subprocess.run(
            optic_cmd,
            timeout=max_time,
            capture_output=True,
            text=True
        )
        execution_time = time.time() - start_time
        if result.returncode == 0:
            # Detectar acciones concurrentes
            concurrent_actions = any(
                concurrency_pattern.search(line)
                for line in result.stdout.split('\n')
            )
            # Guardar el plan en la carpeta planes_generados
            plan_filename = path_planes / f"plan_size_{size}.txt"
            with open(plan_filename, "w") as f:
                f.write(result.stdout)
            print(
                f"Size {size}: Solved in {execution_time:.2f}s | "
                f"Concurrency: {concurrent_actions}"
            )
        else:
            print(f"Planner failed at size {size} with return code {result.returncode}")

    except subprocess.TimeoutExpired:
        print(f"Timeout at size {size}")
    except Exception as e:
        print("Unexpected error:", e)

test_single_problem(size=6, d=6, r=6, l=12, p=12, c=12, g=12)

KeyboardInterrupt: 